In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        arquivo = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(arquivo)

In [ ]:
df.head()

In [ ]:
#verificando como estão os dados do dataframe
df.info()

In [ ]:
#removendo as colunas que não irei precisar
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [ ]:
df.head()

In [ ]:
#veficando como ficou o dataframr do dataframe
df.shape

In [ ]:
#verificando a base de clientes
df.describe()

## Algumas observações sobre a base de clientes.

A média de produtos dos clientes é baixa de apenas 1.53 produtos por cliente, levando em consideração o mix de produtos de uma instituição financeira e o tempo de relacionamento dos clientes com a instituição em que 75% tem relacionamento a mais de 3 anos, podemos considerar que ou a instituição possui um baixo mix de produtos ou realmente os clientes possuem baixa reciprocidade.

Pode-se observar também que a média dos scores e das idades dos clientes é boa, sendo 650 e  38, respectivamente, o que nos leva a inferir que os riscos associados aos clientes dessa instituição é baixo.

Na base temos que 20.37% é o percentual de perda dos clientes, sendo que 51.51% é o percentual de membros ativos e 70.55% dos clientes possuem cartão de crédito.



In [ ]:
#importando os pacotes necessários
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns



# Definir a semente aleatória
random_seed = 42

random.seed(random_seed)
np.random.seed(random_seed)


#separando o target do dataframe
X = df.drop('Exited', axis=1)
y = df['Exited']


# dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# identificando as colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


# criando as transformações para colunas numéricas e categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')

# criando o pré-processamento com ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])





In [ ]:
X_train

In [ ]:
y_train

In [ ]:
# Definir modelos para testar
models = {
    'Logistic Regression': LogisticRegression(random_state=random_seed),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(probability=True, random_state=random_seed),
    'Decision Tree': DecisionTreeClassifier(random_state=random_seed),
    'Random Forest': RandomForestClassifier(random_state=random_seed),
    'Gradient Boosting': GradientBoostingClassifier(random_state=random_seed),
    'XGBoost': XGBClassifier(random_state=random_seed, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(random_state=random_seed)
}

# Avaliar modelos
results = {}
for name, model in models.items():
    # Criar pipeline com pré-processamento e modelo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # Treinar o modelo
    pipeline.fit(X_train, y_train)
    
    # Fazer previsões
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_test, y_prob)
    
    results[name] = {
        'confusion_matrix': cm,
        'classification_report': report,
        'roc_auc': roc_auc
    }

# Comparar resultados
for name, result in results.items():
    print(f"Modelo: {name}")
    print("Matriz de Confusão:")
    print(result['confusion_matrix'])
    print("Relatório de Classificação:")
    print(result['classification_report'])
    print("ROC AUC Score:", result['roc_auc'])
    print("\n")

# Plotar as matrizes de confusão para comparação visual
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
axes = axes.ravel()

for idx, (name, result) in enumerate(results.items()):
    sns.heatmap(result['confusion_matrix'], annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(name)
    axes[idx].set_xlabel('Predição')
    axes[idx].set_ylabel('Verdadeiro')

plt.tight_layout()
plt.show()